In [ ]:
# Imports
#TODO add recursion
#TODO add logging
import os
from docx import Document
from collections import defaultdict
import openpyxl
from openpyxl.styles import Font,PatternFill
import pandas as pd
from collections import defaultdict
not_found = []
none_list= []
exception_list = []
folder_amount_dict = defaultdict(list)
amount_pairs = [
                ("ανέρχεται"," καθίσταται"),
                ("ανέρχεται",")καθίσταται"),
                ("ανερχόμενο","καθίσταται"),
                ('άμεσα',')εντόκως'),
                ('άμεσα','εντόκως'),
                ('άμεσα,',' εντόκως'),
                ('του ανωτέρω ποσού','.'),
                ('ΣΥΝΟΛΙΚΟΥ ΥΨΟΥΣ','εντός'),
                ('στο μέλλον','εντόκως'),
                ('καταβάλλετε άμεσα','εντόκως κατά τα οριζόμενα'),
                ('μεταφορά του χρεωστικού υπολοίπου', 'στον με αριθμό'),
                ("το ποσό των", "εντόκως"),
                ("το ποσό των", ")εντόκως"),
                ('το συνολικά οφειλόμενο ποσό των',""),
                ('το συνολικά οφειλόμενο ποσό των',", πλέον"),
                ("ανερχόμενο",", καθίσταται"),
                ('ΥΨΟΥΣ',""),
                ('ανωτέρω ποσού',""),
                ('ανερχόμενο την', 'πλέον τόκων και εξόδων'),
                ('συνολικού ποσού','εντός'),
                ('ποσό των','ΚΑΛΟΥΜΕ'),
                ('ποσού', 'ληξιπρόθεσμο και απαιτητό'),
                ('στο μέλλον έως το ποσό','ΣΑΣ ΕΠΙΤΑΣΣΟΥΜΕ'),
                ('','')
                  ]



contract_pairs  = [('Με τη με αριθμό','σύμβαση'),('με την υπ’ αριθ.','Σύμβαση'),('με κωδικό πελάτη',''),('υπ αριθμ','σύμβαση'),('υπ αριθμ','συμβάσεις'),('για την παροχή με αριθμό','')]  
# contract_pairs  = [('αριθμό','σύμβαση'),('υπ’ αριθ.','Σύμβαση'),('υπ αριθμ','σύμβαση'),('υπ αριθμ','συμβάσεις')]  


# Excel formatting functions 
def launch_excel(df,name) :
    df.to_excel(f'{name}.xlsx',index=False)
    format_df(f'{name}.xlsx')
    os.startfile(f'{name}.xlsx')

def format_df(file) :
    workbook = openpyxl.load_workbook(file)
    for worksheet in workbook:
        font = Font(color='FFFFFF', bold=True)
        fill = PatternFill(start_color='5552A2', end_color='5552A2', fill_type='solid')
        for cell in worksheet[1]:
            cell.font = font
            cell.fill = fill
        for column in worksheet.columns:
            worksheet.column_dimensions[openpyxl.utils.get_column_letter(column[0].column)].width = 30      
    workbook.save(file)

def indent_item(r, item, index=None):
    relative_path = os.path.relpath(item, r)
    depth = len(relative_path.split(os.sep))
    base_item= os.path.basename(item)
    return(f"{depth * '  '} {index}. {base_item}")

def docstring_simple_flow() :
    '''
    1. search_for_every_amount_contract_pair(directory, amount_pairs, contract_pairs)
        1.1.search_in_files(directory, search_amount_term, search_contract_term, start_amount_string, end_amount_string, start_contract_string, end_contract_string)
            1.2. search_in_docx
                1.3  extract_text(para, target_substring, context_length, start_string=None, end_string=None):
    2. finalise df(found_files)
    '''

def docstring_flow_plus_explanations() :
    '''
    1. search_for_every_amount_contract_pair(directory, amount_pairs, contract_pairs)
        #  Explanation : This function iterates over amount_pairs and contract_pairs, calling search_in_files for each combination.
        1.1.search_in_files(directory, search_amount_term, search_contract_term, start_amount_string, end_amount_string, start_contract_string, end_contract_string)
            Explanation : This function iterates over all files in the given directory, calling search_in_docx for each file.
            1.2. search_in_docx
                Expanation : This function opens a .docx file and searches each paragraph for the specified terms, calling extract_text for matches.
                1.3  extract_text(para, target_substring, context_length, start_string=None, end_string=None):
                    Explanation :This function searches a paragraph for a given substring and optionally cleans the found text based on start and end strings.
    2. finalise df(found_files)
        Explanation : This function processes the found files, organizing the data into a structure suitable for DataFrame creation, and then launches an Excel file with the data.
    '''

def docsttring_found_files() :
    '''
    The structure of `found_files` returned by the function `search_for_every_amount_contract_pair` is a list of tuples. 
    found_files= [(root),(file),tuple_n(amount_snippet,clean_amount_substring,contract_snippet,clean_contract_substring)]
    Example for the first file: 
    1) Get the first files spv folder = found_files[0]
    2) Get the file  = found_files[0][1]
    3) Get the file contents  = found_files[0][2]
    4) Get the first instance of amount and contract  = found_files[0][2][0]
    Get the amount string =   found_files[0][2][0][1]
    5) Get the contract string =   found_files[0][2][0][3]

    Each tuple represents a file where a matching amount and contract pair was found. The structure of each tuple in the `found_files` list is as follows:
    1. **Root Directory (`root`)**: The root directory (anathesi folder) where the file was found. This is a string representing the path to the directory.
    2. **File Path (`file_path`)**: The full path to the file (exodiko doc) where the matching amount and contract pair was found. This is a string.
    3. **Snippets (`snippets`)**: A list of tuples, where each tuple contains information about the found amount and contract within the document. Each tuple in the `snippets` list has the following structure:
    - **Amount Snippet**: A string containing a snippet of text from the document where the search amount term was found, including some context around it.
    - **Clean Amount Substring**: A string extracted from the amount snippet that specifically contains the amount, cleaned according to the start and end amount strings provided.
    - **Contract Snippet**: A string containing a snippet of text from the document where the search contract term was found, including some context around it. If no contract term is found in the same paragraph as the amount term, this will be "No Contract snippet found".
    - **Clean Contract Substring**: A string extracted from the contract snippet that specifically contains the contract number or identifier, cleaned according to the start and end contract strings provided. If no contract term is found in the same paragraph as the amount term, this will be "No Contract number found".
    '''


In [ ]:
import os
from docx import Document
from collections import defaultdict
import pandas as pd

file = r"C:\Users\pallist\Desktop\Desktop\ΤΡΕΧΟΝΤΑ\56 mapping to paths.xlsx"
paths_df = pd.read_excel(file, sheet_name='Paths', dtype=str)
path_list = paths_df['Path'].tolist()

not_found = []
exception_list = []

def extract_text(para, target_substring, context_length, start_string=None, end_string=None):
    start = para.text.find(target_substring)
    end = start + len(target_substring)
    snippet_start = max(0, start - context_length)
    snippet_end = min(len(para.text), end + context_length)
    snippet = para.text[snippet_start:snippet_end]
    cleaned_substring = None
    if start_string and end_string:
        start_target_substring = para.text.find(start_string, snippet_start, snippet_end)
        end_target_substring = para.text.find(end_string, start_target_substring + len(start_string), snippet_end)
        if start_target_substring != -1 and end_target_substring != -1:
            cleaned_substring = para.text[start_target_substring + len(start_string):end_target_substring].strip()
    return snippet, cleaned_substring

def search_in_docx(file_path, search_amount_term, start_amount_string, end_amount_string):
    doc = Document(file_path)
    context_length = 150
    results = []
    for para in doc.paragraphs:
        if search_amount_term in para.text:
            amount_snippet, clean_amount_substring = extract_text(para, search_amount_term, context_length, start_amount_string, end_amount_string)
            results.append((amount_snippet, clean_amount_substring))
    return results

def search_in_files(path_list, search_amount_term, start_amount_string, end_amount_string):
    results = []
    for dir_idx, directory in enumerate(path_list):
        print(f"  {dir_idx+1}.{os.path.basename(directory)} ")
            
        for root_idx, (root, _, files) in enumerate(os.walk(directory)):
            rel_root = os.path.relpath(root, directory)
            depth = len(rel_root.split(os.sep))
            base_r = os.path.basename(root)
            anathesi = root.split(os.sep)[9] if len(root.split(os.sep)) > 9 else base_r
            for file in files:
                file_path = os.path.join(root, file)
                try:
                    if file.endswith('.docx'):
                        snippets = search_in_docx(file_path, search_amount_term, start_amount_string, end_amount_string)
                        if snippets:
                            results.append((anathesi, file_path, snippets, search_amount_term, end_amount_string))
                        else:
                            not_found.append(file_path)
                except Exception as e:
                    exception_list.append(f"Error processing word file {file_path}: {e}")
    return results

def search_for_every_amount_pair(path_list, amount_pairs):
    found_files = []
    for i_amount, amount_pair in enumerate(amount_pairs):
        print(f"{i_amount+1}: {amount_pair}")
        search_amount_term, end_amount_string = amount_pair
        result = search_in_files(path_list, search_amount_term, search_amount_term, end_amount_string)
        found_files += result
        
    return found_files

def finalise_df(found_files):
    unique_id = 0
    total_dict = defaultdict(lambda: ([]))
    amount_tracker = defaultdict(lambda: defaultdict(list))

    for i, found_file in enumerate(found_files):
        main_folder = found_files[i][0]
        file_path = found_files[i][1]
        base_file = os.path.basename(found_file[1])
        search_amount_term = found_files[i][3]
        end_amount_string = found_files[i][4]
        for j, all_instances in enumerate(found_files[i][2]):
            case_count = len(found_files[i][2])
            amount_snippet = all_instances[0]
            amount_number = all_instances[1]
            amount_snippet_used = "Yes" if not amount_number else "No"
            
            # Check for duplicates
            if amount_number in amount_tracker[file_path]:
                original_amount_term, original_end_string = amount_tracker[file_path][amount_number][0]
                duplicate = f"Duplicate: {original_amount_term}-{original_end_string}"
            else:
                amount_tracker[file_path][amount_number].append((search_amount_term, end_amount_string))
                duplicate = ""

            total_dict[unique_id].append([main_folder, file_path, base_file, amount_number, amount_snippet, case_count, search_amount_term, end_amount_string, amount_snippet_used, duplicate])
            unique_id += 1

    data_for_df = [item for sublist in total_dict.values() for item in sublist]
    total_df = pd.DataFrame(data_for_df, columns=['Anathesi', 'File Path', 'File', 'Amount', 'Amount Snippet', 'Cases Count', 'Search Amount Term', 'End Amount String', 'Amount Snippet Used', 'Duplicate'])
    total_df = total_df[total_df['Duplicate'] == '' or total_df['Amount'] == '' ]
    
    return total_df

def print_anatheseis(total_df):
    anatheseis_set = set(total_df['Anathesi'].tolist())
    for i, anathesi in enumerate(anatheseis_set):
        print(f'{i+1}. {anathesi} : {len(total_df[total_df["Anathesi"] == anathesi])} cases')


In [ ]:
found_files = search_for_every_amount_pair(path_list, amount_pairs)
total_df = finalise_df(found_files)
filtered_df = total_df[total_df['Duplicate'].eq('') | total_df['Amount'].eq('')]
filtered_df = total_df.drop_duplicates(subset=['File','Amount'])
filtered_df.to_excel('filtered_files.xlsx',index=False)
os.startfile('filtered_files.xlsx')
